Bhavesh Bhatt

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

# Imports

In [1]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sys import getsizeof

/home/fridh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-03-19 13:18:19.310338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742386699.334228   11058 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742386699.341369   11058 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 13:18:19.365396: I tensorflow/core/platform/cpu_feat

In [2]:
print(tf.__version__)

2.18.0


# Helper Functions

In [3]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [4]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

# Importa Data

In [5]:
import  pandas as pd

df = pd.read_csv("dataset.csv", header=None)              # No header in your format
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float16
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string

In [6]:
dataset = dataset.reshape(dataset.shape[0], 13, 16, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (583554, 13, 16, 1)
Labels shape: (583554,)
Input shape: (13, 16, 1)


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.2, random_state=42, stratify=labels_set)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Print the lengths of the training, validation, and testing sets.
print(f"Training set length: {len(x_train)}")
print(f"Validation set length: {len(x_val)}")
print(f"Testing set length: {len(x_test)}")

Training set length: 373474
Validation set length: 93369
Testing set length: 116711


In [8]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights.
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
dist_class_weight = dict(enumerate(class_weight))

In [9]:
print(f"Classes: {np.unique(labels_set)}")

Classes: ['Background_noise' 'Bus' 'Car' 'Motorcycle' 'Truck']


In [10]:
print(f"Class weights: {dist_class_weight}")

Class weights: {0: np.float64(2.303546536729785), 1: np.float64(0.8562873290458667), 2: np.float64(0.8566506869738744), 3: np.float64(0.9921999946866449), 4: np.float64(0.8177576334833208)}


In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))
y_val = to_categorical(label_encoder.fit_transform(y_val))

## Build & Compile the model


In [12]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

model = keras.Sequential([
    InputLayer(shape=input_shape),  # 25 frames, 16 channels

    SeparableConv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    SeparableConv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),

    # Feature Pooling (Combining Max & Average Pooling)
    GlobalAveragePooling2D(),

    # Fully Connected Layer
    Dense(32, activation="relu", kernel_regularizer=l2(0.001)),
    Dense(16, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.4),  # Dropout slightly reduced for stability

    # Output Layer (Softmax for multi-class classification)
    Dense(units=len(np.unique(labels_set)))
])

I0000 00:00:1742386715.484402   11058 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4201 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ separable_conv2d                │ (None, 13, 16, 64)     │           137 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 13, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 6, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 6, 8, 64)       │         4,736 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 6, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 3, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,078 (31.55 KB)

 Trainable params: 7,822 (30.55 KB)

 Non-trainable params: 256 (1.00 KB)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy']
)

In [16]:
model.fit(x_train, y_train, epochs=4, validation_data=(x_val, y_val), class_weight=dist_class_weight, batch_size=64)

Epoch 1/4


I0000 00:00:1742386728.002176   11139 service.cc:148] XLA service 0x7b403c010310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742386728.002212   11139 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2025-03-19 13:18:48.079327: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742386728.378367   11139 cuda_dnn.cc:529] Loaded cuDNN version 90300


  25/5836 ━━━━━━━━━━━━━━━━━━━━ 40s 7ms/step - categorical_accuracy: 0.2136 - loss: 1.9877 

I0000 00:00:1742386731.173076   11139 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5836/5836 ━━━━━━━━━━━━━━━━━━━━ 52s 8ms/step - categorical_accuracy: 0.3499 - loss: 1.3636 - val_categorical_accuracy: 0.3866 - val_loss: 1.3447
Epoch 2/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 39s 7ms/step - categorical_accuracy: 0.3935 - loss: 1.2367 - val_categorical_accuracy: 0.4068 - val_loss: 1.3099
Epoch 3/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 38s 7ms/step - categorical_accuracy: 0.4015 - loss: 1.2122 - val_categorical_accuracy: 0.4210 - val_loss: 1.2824
Epoch 4/4
5836/5836 ━━━━━━━━━━━━━━━━━━━━ 37s 6ms/step - categorical_accuracy: 0.4078 - loss: 1.1986 - val_categorical_accuracy: 0.4072 - val_loss: 1.3243


In [17]:
KERAS_MODEL_NAME = "tf_model_mini.h5"

In [18]:
model.save(KERAS_MODEL_NAME)

In [19]:
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")

File size: 0.156 Megabytes


In [20]:
keras_model_size = get_file_size(KERAS_MODEL_NAME)

In [21]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy is {}%'.format(round(100*test_acc, 2)))

3648/3648 - 17s - 5ms/step - categorical_accuracy: 0.4097 - loss: 1.3271

Test accuracy is 40.97%


# TF Lite Model

In [22]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [23]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmppne5pk6c/assets


INFO:tensorflow:Assets written to: /tmp/tmppne5pk6c/assets


Saved artifact at '/tmp/tmppne5pk6c'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13, 16, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  135523384691872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523386080448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384689936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384698736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384700144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384697680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384698208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384697504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384701376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384701728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135523384703488:

W0000 00:00:1742386911.542371   11058 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742386911.542393   11058 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-19 13:21:51.542781: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmppne5pk6c
2025-03-19 13:21:51.544280: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-19 13:21:51.544296: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmppne5pk6c
I0000 00:00:1742386911.555158   11058 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-03-19 13:21:51.557100: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-19 13:21:51.616117: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmppne5pk6c
2025-03-19 13:21:51.633742: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 

In [24]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

37328

In [25]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 36.453 Kilobytes


In [26]:
tflite_file_size = get_file_size(TF_LITE_MODEL_FILE_NAME)

# Check Input Tensor Shape

In [27]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 13 16  1]
Input Type: <class 'numpy.float32'>
Output Shape: [1 5]
Output Type: <class 'numpy.float32'>


In [28]:
# Load your existing model
model = tf.keras.models.load_model('tf_model_mini.h5')

# Convert the model to TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.uint8]

# Ensure that the input and output types are uint8
def representative_dataset_gen():
    for i in range(len(x_test)):
        # Get sample input data as a numpy array in a method of your choosing
        yield [x_test[i]]

converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()

# Save the quantized model
with open('pipeline_int8_model_data.tflite', 'wb') as f:
    f.write(tflite_quant_model)


INFO:tensorflow:Assets written to: /tmp/tmpvft4cnsf/assets


INFO:tensorflow:Assets written to: /tmp/tmpvft4cnsf/assets


Saved artifact at '/tmp/tmpvft4cnsf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 13, 16, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  135522732238528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732243984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732281872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732289792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732290496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732286448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732288736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732363968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732365024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732364144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135522732371008: 

W0000 00:00:1742386913.144739   11058 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742386913.144757   11058 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-19 13:21:53.144970: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvft4cnsf
2025-03-19 13:21:53.146056: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-19 13:21:53.146072: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpvft4cnsf
2025-03-19 13:21:53.158438: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-19 13:21:53.221655: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpvft4cnsf
2025-03-19 13:21:53.239164: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 94195 microseconds.


In [29]:
# Load and test the quantized model
interpreter = tf.lite.Interpreter(model_path='quantized_model.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the TensorFlow Lite model on random data from x_test
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

# Test the TensorFlow Lite model on random data from x_test
input_data = x_test[np.random.choice(x_test.shape[0], 1)]
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

print("TensorFlow Lite model output:", tflite_results)

ValueError: Could not open 'quantized_model.tflite'.

# Resize Tensor Shape

In [ ]:
x_test = x_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [29]:
interpreter.resize_tensor_input(input_details[0]['index'], x_test.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [999  13  16   1]
Input Type: <class 'numpy.float32'>
Output Shape: [231   5]
Output Type: <class 'numpy.float32'>


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [30]:
x_test.dtype

dtype('float32')

In [31]:
test_imgs_numpy = np.array(x_test, dtype=np.float32)

In [ ]:
test_imgs_numpy.dtype

dtype('float32')

: 

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
# Convert y_test from one-hot encoded format to label format
y_test_labels = np.argmax(y_test, axis=1)[:len(prediction_classes)]
acc = accuracy_score(prediction_classes, y_test_labels)

In [ ]:
print('Test accuracy TFLITE model is {}%'.format(round(100*acc, 2)))

In [ ]:
tflite_file_size/keras_model_size

# TF Lite Model Float 16

In [ ]:
TF_LITE_MODEL_FLOAT_16_FILE_NAME = "tf_lite_float_16_model.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = TF_LITE_MODEL_FLOAT_16_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
convert_bytes(get_file_size(TF_LITE_MODEL_FLOAT_16_FILE_NAME), "KB")

In [ ]:
tflite_float_16_file_size = get_file_size(TF_LITE_MODEL_FLOAT_16_FILE_NAME)

In [ ]:
tflite_float_16_file_size/keras_model_size

In [ ]:
tflite_float_16_file_size/tflite_file_size

# TF Lite Size Quantized

In [ ]:
TF_LITE_SIZE_QUANT_MODEL_FILE_NAME = "tf_lite_quant_model.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = tf_lite_converter.convert()

In [ ]:
tflite_model_name = TF_LITE_SIZE_QUANT_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
convert_bytes(get_file_size(TF_LITE_SIZE_QUANT_MODEL_FILE_NAME), "KB")

In [ ]:
tflite_float_quant_file_size = get_file_size(TF_LITE_SIZE_QUANT_MODEL_FILE_NAME)

In [ ]:
tflite_float_quant_file_size/keras_model_size

In [ ]:
tflite_float_quant_file_size/ tflite_float_16_file_size

# Accuracy of the Quantized Model

# Check Input Tensor Shape

In [ ]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_SIZE_QUANT_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

# Resize Tensor Shape

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], x_test.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
x_test.dtype

In [ ]:
test_imgs_numpy = np.array(x_test, dtype=np.float32)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
# Convert y_test from one-hot encoded format to label format
y_test_labels = np.argmax(y_test, axis=1)[:len(prediction_classes)]
acc = accuracy_score(prediction_classes, y_test_labels)

In [ ]:
print('Test accuracy TFLITE Quantized model is {}%'.format(round(100*acc, 2)))